# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import os
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

exp=Experiment(workspace = ws, name = "HyperDriveRun")
project_folder = './capstone-project'

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
# Retrieve data from datasets
# Retrieve data from datasets
key = "CCFraud"
description_text = "Credit card fraud dataset"

try:
    dataset = ws.datasets[key]
except:
    print("Dataset not found in Azure datasets.") 


df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
cluster_name = "Capstone-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
early_termination_policy = MedianStoppingPolicy(delay_evaluation = 5, evaluation_interval = 1)

#Specifying hyper parameters to test
param_sampling = RandomParameterSampling(
    {
        '--bootstrap' : choice(True, False),
        '--max_depth': choice(10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None),
        '--max_features': choice()'auto', 'sqrt'), 
        '--min_samples_leaf': choice(1, 2, 4),
        '--min_samples_split': choice(2, 5, 10),
        '--n_estimators': (200, 400, 600, 800, 1000)
    }
)

#Creating environment 
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory="./",
                      script='train.py',
                      arguments=None,
                      compute_target=compute_target,
                      environment=sklearn_env)

#Detailing run configuration
hyperdrive_run_config = HyperDriveConfig(
    run_config = src,
    hyperparameter_sampling = param_sampling,
    policy = early_termination_policy, 
    primary_metric_name = 'Auc weighted',
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs =  16,
    max_concurrent_runs = 4)

In [ ]:
#Submitting experiment
remote_run = exp.submit(config = hyperdrive_run_config)

## Run Details

In [ ]:
#Getting run details
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#Obtaining details of best model from Hyper Drive run
best_run = remote_run.get_best_run_by_primary_metric(top = 1)
print(best_run.get_details())
remote_run.get_children_sorted_by_primary_metric(top=1)

In [ ]:
#Saving best model from Hypder Drive run
model = hyperdrive_run.register_model(model_name='HyperDriveModel', model_path='outputs/model.pkl')

In [ ]:
# Clean up and shut down compute cluster
compute_target.delete()

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

